In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import minmax_scale

In [2]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_predict = "F:/178/Tugas Akhir/dataset/predict/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/dataset/predict/"

In [3]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.25,random_state=666)
    
    return train_data, test_data, train_values, test_values

def training(data,values,clf):
    trained = clf.fit(data,values)
    
    return trained
    
def predict(test,true,clf):
    pred = clf.predict(test)
    df = pd.DataFrame(data=[pred,true])
    
    return pred,df

def score(pred,true):
    r2 = r2_score(true, pred)
    
    return r2

def crosval(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

def save_model(model,case,types):
    path = path_to_model+types+case+'.sav' # save the model
    joblib.dump(model, path)

def save_predict(df1,df2,case):
    path = path_to_predict+case+".xlsx" # save the model
    arousal = df1.T
    valence = df2.T
    merged = arousal.merge(valence, how='outer', left_index=True, right_index=True)
    merged.to_excel(path,header=None,index=None)

def norm(data):
    scaler.fit(data)
    normalized = scaler.transform(data)
    
    return normalized

In [4]:
kernels = RationalQuadratic(length_scale=1.0, alpha=1.0, 
                            length_scale_bounds=(1e-6, 1e6), 
                            alpha_bounds=(1e-6, 1e6))

# kernels = RBF(length_scale=1.0, length_scale_bounds=(1e-05, 100000.0))
# kernels = ExpSineSquared(length_scale=1.0, periodicity=1.0, 
#                          length_scale_bounds=(1e-07, 10000000.0), 
#                          periodicity_bounds=(1e-07, 10000000.0))

# CASE 1

In [5]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

#split arousal
train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar)
#split valence
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) 

(744, 26)


INITIAL MODEL

In [6]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Arousal

# gpr_a = GPR(kernel=kernels,normalize_y=True,random_state=1000,alpha=0.1)
# gpr_v = GPR(kernel=kernels,normalize_y=True,random_state=1000,alpha=0.1)

In [7]:
print("arousal:\t",gpr_a.kernel)
print("arousal:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
arousal:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [8]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [9]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [10]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0138, length_scale=0.507)
valence:	 RationalQuadratic(alpha=0.017, length_scale=0.159)


AROUSAL

In [11]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.03,0.25,-0.11,-0.37,-0.16,0.17,-0.34,-0.37,-0.47,0.24,...,0.49,0.23,-0.32,0.26,0.34,0.25,-0.18,-0.37,-0.49,-0.33
1,-0.21,-0.26,-0.56,-0.47,-0.50,0.29,-0.41,-0.24,-0.62,-0.21,...,0.68,0.26,-0.09,0.65,0.29,0.35,0.21,-0.65,-0.35,-0.21


R2 Score Arousal:	0.56


VALENCE

In [12]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.14,0.13,-0.02,-0.17,0.12,-0.13,-0.12,-0.13,-0.26,0.31,...,0.33,0.26,-0.09,0.10,0.18,0.36,0.06,-0.19,-0.21,-0.16
1,0.08,-0.57,-0.51,-0.54,-0.20,-0.02,-0.02,-1.00,-0.11,0.11,...,0.17,0.63,0.32,-0.05,0.57,0.08,0.32,-0.51,-0.08,0.02


R2 Score Valence: 0.25


REGRESSION

In [13]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [14]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [15]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [16]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 2

In [17]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 8)


INITIAL MODEL

In [18]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Valence

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Valence

# # gpr_a = GPR()
# # gpr_v = GPR()

In [19]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [20]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [21]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [22]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.00587, length_scale=0.00047)
valence:	 RationalQuadratic(alpha=0.00441, length_scale=1e-06)


AROUSAL

In [23]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.10,0.41,-0.14,-0.51,-0.14,0.06,0.14,-0.27,-0.15,0.25,...,0.40,0.01,-0.17,0.02,0.20,0.04,-0.22,-0.36,-0.14,-0.44
1,-0.21,-0.26,-0.56,-0.47,-0.50,0.29,-0.41,-0.24,-0.62,-0.21,...,0.68,0.26,-0.09,0.65,0.29,0.35,0.21,-0.65,-0.35,-0.21


R2 Score Arousal:	0.38


VALENCE

In [24]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.01,0.13,-0.07,-0.24,0.04,0.12,0.14,0.02,-0.07,0.20,...,0.10,0.07,0.02,-0.01,0.22,0.18,-0.02,-0.16,0.04,-0.19
1,0.08,-0.57,-0.51,-0.54,-0.20,-0.02,-0.02,-1.00,-0.11,0.11,...,0.17,0.63,0.32,-0.05,0.57,0.08,0.32,-0.51,-0.08,0.02


R2 Score Valence:	0.18


REGRESSION

In [25]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [26]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [27]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [28]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 3

In [29]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 3)


INITAL MODEL

In [30]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Valence

# gpr_a = GPR()
# gpr_v = GPR()

In [31]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [32]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [33]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [34]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0058, length_scale=1e-06)
valence:	 RationalQuadratic(alpha=0.00606, length_scale=1e-06)


AROUSAL

In [35]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.13,0.01,0.04,-0.36,-0.0,0.14,-0.37,0.03,-0.30,0.18,...,0.02,0.19,-0.03,0.18,-0.00,0.19,-0.02,-0.04,-0.12,-0.02
1,-0.21,-0.26,-0.56,-0.47,-0.5,0.29,-0.41,-0.24,-0.62,-0.21,...,0.68,0.26,-0.09,0.65,0.29,0.35,0.21,-0.65,-0.35,-0.21


R2 Score Arousal:	0.18


VALENCE

In [36]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.17,0.08,0.05,-0.11,0.13,-0.06,-0.19,0.1,-0.08,0.19,...,0.05,0.16,0.09,0.18,0.11,0.19,-0.02,0.10,0.02,0.18
1,0.08,-0.57,-0.51,-0.54,-0.20,-0.02,-0.02,-1.0,-0.11,0.11,...,0.17,0.63,0.32,-0.05,0.57,0.08,0.32,-0.51,-0.08,0.02


R2 Score Valence:	0.07


REGRESSION

In [37]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [38]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [39]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [40]:
save_predict(df_a,df_v,datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 4

In [41]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 24)


INITIAL MODEL

In [42]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False,  
                    random_state=1000) #init Valence

# # gpr_a = GPR()
# # gpr_v = GPR()

In [43]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [44]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [45]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [46]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0156, length_scale=0.336)
valence:	 RationalQuadratic(alpha=0.0148, length_scale=0.159)


AROUSAL

In [47]:
# # df4_a = df4_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.081422,-0.361852,-0.152641,-0.025052,-0.307071,0.058601,-0.164520,0.031634,0.120427,0.139953,...,0.055981,0.102038,0.202022,0.202652,0.211087,-0.007445,0.285772,0.060685,-0.353485,-0.280892
1,-0.205882,-0.264706,-0.558824,-0.470588,-0.500000,0.294118,-0.411765,-0.235294,-0.617647,-0.205882,...,0.676471,0.264706,-0.088235,0.647059,0.294118,0.352941,0.205882,-0.647059,-0.352941,-0.205882


R2 Score Arousal:	0.28


VALENCE

In [48]:
# # df4_v = df4_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.118480,-0.132967,-0.133773,0.064050,-0.117564,0.123432,-0.009836,0.089491,0.138527,0.234036,...,0.183200,0.158774,0.170037,0.253165,0.223017,0.235391,0.249505,0.168893,-0.083331,-0.061661
1,0.076923,-0.569231,-0.507692,-0.538462,-0.200000,-0.015385,-0.015385,-1.000000,-0.107692,0.107692,...,0.169231,0.630769,0.323077,-0.046154,0.569231,0.076923,0.323077,-0.507692,-0.076923,0.015385


R2 Score Valence:	0.10


REGRESSION

In [49]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [50]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [51]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [52]:
save_predict(df_a,df_v,datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 5

In [53]:
datafitur = "case5" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

#split arousal
train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar)
#split valence
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) 

(744, 34)


In [54]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Arousal

In [55]:
print("arousal:\t",gpr_a.kernel)
print("arousal:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
arousal:	 RationalQuadratic(alpha=1, length_scale=1)


In [56]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [57]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [58]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0138, length_scale=0.595)
valence:	 RationalQuadratic(alpha=0.0183, length_scale=0.238)


In [59]:
# df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.041094,0.311114,-0.107564,-0.385560,-0.132587,0.169879,-0.287253,-0.368045,-0.486673,0.204376,...,0.507992,0.178927,-0.377253,0.212966,0.342567,0.169717,-0.152999,-0.383854,-0.454043,-0.374128
1,-0.205882,-0.264706,-0.558824,-0.470588,-0.500000,0.294118,-0.411765,-0.235294,-0.617647,-0.205882,...,0.676471,0.264706,-0.088235,0.647059,0.294118,0.352941,0.205882,-0.647059,-0.352941,-0.205882


R2 Score Arousal:	0.53


In [60]:
# df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.158909,0.155287,-0.064628,-0.195087,0.113134,-0.103232,-0.070748,-0.076047,-0.274730,0.274158,...,0.311702,0.250663,-0.151236,0.044411,0.216046,0.330457,0.065228,-0.208555,-0.193520,-0.188528
1,0.076923,-0.569231,-0.507692,-0.538462,-0.200000,-0.015385,-0.015385,-1.000000,-0.107692,0.107692,...,0.169231,0.630769,0.323077,-0.046154,0.569231,0.076923,0.323077,-0.507692,-0.076923,0.015385


R2 Score Valence: 0.23


In [61]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [62]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [63]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 6

In [64]:
datafitur = "case6" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape) 

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 29)


In [65]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Valence

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Valence

In [66]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


In [67]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [68]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [69]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0148, length_scale=0.611)
valence:	 RationalQuadratic(alpha=0.0185, length_scale=0.225)


In [70]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.03,0.26,-0.11,-0.42,-0.2,0.17,-0.28,-0.38,-0.51,0.25,...,0.51,0.23,-0.35,0.27,0.36,0.21,-0.13,-0.37,-0.52,-0.36
1,-0.21,-0.26,-0.56,-0.47,-0.5,0.29,-0.41,-0.24,-0.62,-0.21,...,0.68,0.26,-0.09,0.65,0.29,0.35,0.21,-0.65,-0.35,-0.21


R2 Score Arousal:	0.55


In [71]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.11,0.14,-0.03,-0.14,0.12,-0.14,-0.08,-0.17,-0.27,0.32,...,0.35,0.28,-0.11,0.07,0.20,0.33,0.12,-0.19,-0.20,-0.18
1,0.08,-0.57,-0.51,-0.54,-0.20,-0.02,-0.02,-1.00,-0.11,0.11,...,0.17,0.63,0.32,-0.05,0.57,0.08,0.32,-0.51,-0.08,0.02


R2 Score Valence:	0.25


In [72]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [73]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [74]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [75]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 7

In [76]:
datafitur =  "case7" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 50)


In [77]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Valence

In [78]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


In [79]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [80]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [81]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0187, length_scale=2.46)
valence:	 RationalQuadratic(alpha=0.0238, length_scale=1.27)


In [82]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.16,-0.20,-0.26,-0.37,-0.25,0.22,-0.38,-0.31,-0.58,0.32,...,0.39,0.14,-0.25,0.54,0.33,0.34,0.02,-0.37,-0.40,-0.25
1,-0.21,-0.26,-0.56,-0.47,-0.50,0.29,-0.41,-0.24,-0.62,-0.21,...,0.68,0.26,-0.09,0.65,0.29,0.35,0.21,-0.65,-0.35,-0.21


R2 Score Arousal:	0.61


In [83]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.07,-0.06,-0.24,-0.19,-0.04,0.07,-0.15,-0.04,-0.31,0.47,...,0.26,0.19,-0.01,0.33,0.36,0.39,0.16,-0.16,-0.17,-0.04
1,0.08,-0.57,-0.51,-0.54,-0.20,-0.02,-0.02,-1.00,-0.11,0.11,...,0.17,0.63,0.32,-0.05,0.57,0.08,0.32,-0.51,-0.08,0.02


R2 Score Valence:	0.26


In [84]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [85]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [86]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [87]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 8

In [88]:
datafitur =  "case8" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 37)


In [89]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Valence

In [90]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


In [91]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [92]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [93]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0138, length_scale=0.617)
valence:	 RationalQuadratic(alpha=0.0193, length_scale=0.292)


In [94]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.05,0.32,-0.11,-0.43,-0.16,0.18,-0.27,-0.38,-0.51,0.21,...,0.51,0.19,-0.40,0.22,0.35,0.17,-0.12,-0.38,-0.48,-0.39
1,-0.21,-0.26,-0.56,-0.47,-0.50,0.29,-0.41,-0.24,-0.62,-0.21,...,0.68,0.26,-0.09,0.65,0.29,0.35,0.21,-0.65,-0.35,-0.21


R2 Score Arousal:	0.54


In [95]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,-0.14,0.17,-0.07,-0.15,0.12,-0.10,-0.08,-0.09,-0.28,0.28,...,0.33,0.26,-0.16,0.03,0.23,0.31,0.11,-0.21,-0.19,-0.20
1,0.08,-0.57,-0.51,-0.54,-0.20,-0.02,-0.02,-1.00,-0.11,0.11,...,0.17,0.63,0.32,-0.05,0.57,0.08,0.32,-0.51,-0.08,0.02


R2 Score Valence:	0.23


In [96]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [97]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [98]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [99]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 9

In [100]:
datafitur =  "case9" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 58)


In [101]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Valence

In [102]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


In [103]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [104]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [105]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0196, length_scale=2.9)
valence:	 RationalQuadratic(alpha=0.0229, length_scale=1.26)


In [106]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.11,-0.19,-0.28,-0.34,-0.26,0.19,-0.35,-0.30,-0.57,0.39,...,0.42,0.10,-0.26,0.49,0.35,0.32,-0.03,-0.34,-0.38,-0.27
1,-0.21,-0.26,-0.56,-0.47,-0.50,0.29,-0.41,-0.24,-0.62,-0.21,...,0.68,0.26,-0.09,0.65,0.29,0.35,0.21,-0.65,-0.35,-0.21


R2 Score Arousal:	0.61


In [107]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.02,-0.05,-0.28,-0.21,-0.05,0.06,-0.12,-0.03,-0.29,0.46,...,0.28,0.16,0.01,0.31,0.38,0.37,0.11,-0.14,-0.16,-0.07
1,0.08,-0.57,-0.51,-0.54,-0.20,-0.02,-0.02,-1.00,-0.11,0.11,...,0.17,0.63,0.32,-0.05,0.57,0.08,0.32,-0.51,-0.08,0.02


R2 Score Valence:	0.26


In [108]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [109]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [110]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [111]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 10

In [112]:
datafitur =  "case10" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 61)


In [113]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=False, copy_X_train=False, 
                    random_state=1000) #init Valence

In [114]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


In [115]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [116]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [117]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 RationalQuadratic(alpha=0.0198, length_scale=2.98)
valence:	 RationalQuadratic(alpha=0.0223, length_scale=1.21)


In [118]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.11,-0.20,-0.27,-0.37,-0.26,0.20,-0.31,-0.28,-0.58,0.40,...,0.43,0.10,-0.25,0.48,0.36,0.32,-0.02,-0.34,-0.39,-0.27
1,-0.21,-0.26,-0.56,-0.47,-0.50,0.29,-0.41,-0.24,-0.62,-0.21,...,0.68,0.26,-0.09,0.65,0.29,0.35,0.21,-0.65,-0.35,-0.21


R2 Score Arousal:	0.60


In [119]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.02,-0.05,-0.28,-0.21,-0.05,0.06,-0.12,-0.02,-0.30,0.46,...,0.29,0.16,0.01,0.30,0.38,0.37,0.11,-0.14,-0.14,-0.06
1,0.08,-0.57,-0.51,-0.54,-0.20,-0.02,-0.02,-1.00,-0.11,0.11,...,0.17,0.63,0.32,-0.05,0.57,0.08,0.32,-0.51,-0.08,0.02


R2 Score Valence:	0.26


In [120]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [121]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [122]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [123]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
# save_model(trained_a,datafitur,'arousal/') #save model Arousal
# save_model(trained_v,datafitur,'valence/') #save model Valence